## 0 - Duplicate and name per-module notebook.

- In panel at left, select this notebook ("InitiaProcedures-CIT.ipynb"), and Duplicate it.
- Rename the notebook, replacing "CIT-Copy1" with the module's name (e.g. "SC42")
- Close this notebook to avoid confusion, open the module one.

In [ ]:
%matplotlib inline
from importlib import reload
import numpy as np
import pathlib
import matplotlib.pyplot as plt

from ics.cobraCharmer import pfiDesign
from ics.cobraCharmer import cobraState

from ics.cobraCharmer.utils import butler
from procedures.moduleTest import moduleTest, calculation, ontimeModel, plotUtils


## 1 - Bootstrap

Takes the initial XML map from the assembly bench and makes it functional
 - does a frequency calibratikon on both motors (separately).
 - assigns starting center positions from the found fiber spots.
 - initializes some parts of the geometry, clears the rest.
 
Note that bootstrapping *requires* that the the cobras be in the theta-out, phi-in safe positions. Due to the
frequency calibration the theta positions will need to be manually restored after bootstrapping. 

The input map, for the example module named "SC42", "SC42_init.xml", and comes from the assembly bench procedures. The output map is "SC42_bootstrap.xml". This is then used for the next step.

In [ ]:
moduleName = "SC42"

In [ ]:
from procedures.cit import bootstrapModule

# CPL: Get the ordering from the "site" module.
bootstrapMap = bootstrapModule.bootstrapModule(moduleName, numberCobrasFromRight=True)

## 2 - Please manually reset theta motors

Sorry, but the frequency calibration almost certainly moved (only) the theta motors. Please manually put them back to the "out" position.


## 3 - Make phi motor maps at several ontimes; create working phi map

Takes a set of phi motor maps at a range of ontimes, then generates a new map with tuned slow and fast maps.

Since we are only looking for overall ontimes, we make coarse (200-step) runs.

The initial map is named "SC42_bootstrap.xml", and the final output is "SC42_phiOntime.xml".

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='bootstrap'))

In [ ]:
%pdb on
phiSteps = 100
phiRuns = dict()
for phiOntime in 65,50,30,20,15:
    outputDir = mt.makePhiMotorMap(f'phi_{phiOntime}ms.xml', phiOnTime=phiOntime/1000, 
                                   updateGeometry=True, repeat=1, fast=False, steps=phiSteps)
    phiRuns[phiOntime] = outputDir
%pdb off

## 4 - Choose slowest working phi map

Using the phi ontime scans we just made, chose the slowest one for cobra which run limit-to-limit.

CPL -- This is _far_ from  optimal. 

In [ ]:
reload(ontimeModel)
poptmap = ontimeModel.ontimeModel.loadFromPhiData(phiRuns[65], phiRuns)
fwot, rvot = poptmap.getSlowestGoodOntimes(closeEnough=np.deg2rad(1))
poptmap.saveNewMap(butler.mapPathForModule(moduleName, 'phiOntime'))
print(fwot)
print(rvot)

## 4a - Test new phi map

In [ ]:
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiOntime'))
mt.setPhiGeometryFromRun(phiRuns[65])

In [ ]:
phiGeometryRun = mt.makePhiMotorMap(f'phiFinal.xml', updateGeometry=True, repeat=1, fast=False, steps=phiSteps)
phiRuns[999] = phiGeometryRun
butler.publishMapForModule(moduleName, version='phiFinal', fromRunPath=phiGeometryRun)

In [ ]:
_ = plotUtils.plotOntimeSet(moduleName, phiRuns, 'phi', phiSteps)

## 5 - Run phi convergence test.

At this point we know the phi motor center position, and we have a decent map. So run the phi convergence test now.
Note that we are applying Erin's heuristic, and dynamically scaling each motor's ontime with each move.

We are only trying for 10 mrad, or ~25 um. Off by a factor of 5.
Also, stay ~5 degrees away from ends, 

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiFinal'))
mt.setPhiGeometryFromRun(phiRuns[65])

In [ ]:
rstate = np.random.RandomState(2394)
angles = rstate.uniform(5,175,100)

# Force us to finish at 60, just for convenience.
angles[-1] = 60.0

# Start with same medium angles, to initialize the scaling
angles[0] = 60.0
angles[1] = 30.0
angles[2] = 90.0

tolerance = np.rad2deg(0.01)

phiConvergenceRuns = []
for a_i, a in enumerate(angles):
    runDir = mt.moveToPhiAngle(angle=a, tolerance=tolerance, 
                               keepExistingPosition=(a_i > 0), maxTries=8)
    phiConvergenceRuns.append(runDir)

In [ ]:
plotUtils.plotConvergenceRuns()

## 6 - Make theta maps at several ontimes; create theta motor map

Takes a set of theta motor maps at a range of ontimes, then generates a new map with tuned slow and fast maps.

The initial map is named "SC42_thetaOntime.xml" or "SC42.xml", and the output is "SC42_theta.xml"

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiFinal'))

In [ ]:
thetaRuns = dict()
thetaSteps = dict()
for thetaOntime in 70,60,50,40,30:
    outputDir = mt.makeThetaMotorMap(f'theta_{thetaOntime}ms.xml', thetaOnTime=thetaOntime/1000, 
                                     phiRunDir=phiGeometryRun,
                                     updateGeometry=True, repeat=1, fast=False, steps=200)
    thetaSteps[thetaOntime] = thetaSteps
    thetaRuns[thetaOntime] = outputDir

## 7 - Choose slowest working theta map

In [ ]:
reload(ontimeModel)
%pdb on
toptmap = ontimeModel.ontimeModel.loadFromThetaData(thetaRuns[60], thetaRuns)
tfwot, trvot = toptmap.getSlowestGoodOntimes(closeEnough=np.deg2rad(1))
toptmap.saveNewMap(butler.mapPathForModule(moduleName, 'thetaOntime'))
%pdb off
print(tfwot)
print(trvot)

## 7a - Test new map

In [ ]:
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='thetaOntime'))
mt.setThetaGeometryFromRun(thetaRuns[60])

In [ ]:
thetaGeometryRun = mt.makeThetaMotorMap(f'thetaFinal.xml', phiRunDir=phiGeometryRun, updateGeometry=True, repeat=1, fast=False, steps=100)
thetaRuns[999] = thetaGeometryRun
thetaSteps[999] = 100
butler.publishMapForModule(moduleName, version='thetaFinal', fromRunPath=thetaGeometryRun)

In [ ]:
reload(plotUtils)
_ = plotUtils.plotOntimeSet(moduleName, thetaRuns, 'theta', thetaSteps)

## 8 - Run theta convergence test

In [ ]:
thetaGeometryRun = thetaRuns[60]

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='thetaOntime'))
mt.setThetaGeometryFromRun(thetaGeometryRun)

In [ ]:
%pdb on
rstate = np.random.RandomState(2394)
angles = rstate.uniform(1,370,100)

#Start with medium slews, to initialize the scaling
angles[0] = 60.0
angles[1] = 30.0
angles[2] = 90.0

thetaConvergenceRuns = []
for a_i, a in enumerate(angles):
    ret = mt.moveToThetaAngle(angle=a, tolerance=np.rad2deg(0.01), 
                              keepExistingPosition=(a_i > 0), maxTries=8, scaleFactor=10)
    thetaConvergenceRuns.append(ret)
%pdb off


In [ ]:
reload(plotUtils)
%pdb on
ret = plotUtils.plotConvergenceRuns(tconvRuns, 'theta')
%pdb off


In [ ]:
ret[0].savefig(tconvRuns[-1] / 'output' / f'{moduleName}_thetaConvergence.pdf')

## 9 - Return cobras to safe position

In [ ]:
mt.gotoSafeFromPhi60()